In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_olivetti_faces
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_score, jaccard_similarity_score
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from subprocess import call
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn import decomposition
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn. model_selection import RandomizedSearchCV
from sklearn. model_selection import GridSearchCV

# Hunting Pemain Basket

In [2]:
df = pd.read_csv('all_seasons.csv')
df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Chris Robinson,VAN,23.0,195.58,90.71840,Western Kentucky,USA,1996,2,...,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97
1,1,Matt Fish,MIA,27.0,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,2,Matt Bullard,HOU,30.0,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,3,Marty Conlon,BOS,29.0,210.82,111.13004,Providence,USA,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,4,Martin Muursepp,DAL,22.0,205.74,106.59412,None,USA,1996,1,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [3]:
df.columns

Index(['Unnamed: 0', 'player_name', 'team_abbreviation', 'age',
       'player_height', 'player_weight', 'college', 'country', 'draft_year',
       'draft_round', 'draft_number', 'gp', 'pts', 'reb', 'ast', 'net_rating',
       'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'],
      dtype='object')

In [4]:
df
df = df[['player_name', 'age', 'player_height', 'player_weight', 'pts', 'reb']]
df['target'] = df.apply(lambda x: 1 if (x['age']<=25) & (x['player_height']>=180) & (x['player_weight']<=90) & (x['pts']>=6) & (x['reb']>=3) else 0, axis=1)
# df.shape
df

,player_name,age,player_height,player_weight,pts,reb,target
0,Chris Robinson,23.0,195.58,90.71840,4.6,1.7,0
1,Matt Fish,27.0,210.82,106.59412,0.3,0.8,0
2,Matt Bullard,30.0,208.28,106.59412,4.5,1.6,0
3,Marty Conlon,29.0,210.82,111.13004,7.8,4.4,0
4,Martin Muursepp,22.0,205.74,106.59412,3.7,1.6,0
...,...,...,...,...,...,...,...
9556,Elijah Millsap,29.0,200.66,97.52228,1.5,3.0,0
9557,Elfrid Payton,23.0,193.04,83.91452,12.8,4.7,1
9558,Edy Tavares,25.0,220.98,120.20188,4.0,5.5,0
9559,Willy Hernangomez,23.0,210.82,108.86208,8.2,7.0,0


In [5]:
df.isnull().sum()

player_name      0
age              0
player_height    0
player_weight    0
pts              0
reb              0
target           0
dtype: int64

### Standarisasi

In [6]:
stdscaler = StandardScaler()

datastd = stdscaler.fit_transform(df.iloc[:,1:6])
datastd

array([[-0.99674183, -0.57899275, -0.80260413, -0.58938997, -0.74785002],
       [-0.07727165,  1.06312962,  0.43960663, -1.31961423, -1.10871173],
       [ 0.61233098,  0.78944256,  0.43960663, -0.60637193, -0.78794577],
       ...,
       [-0.53700674,  2.15787786,  1.50435871, -0.69128173,  0.7757883 ],
       [-0.99674183,  1.06312962,  0.61706531,  0.02196056,  1.37722448],
       [ 0.61233098,  0.24206844,  0.08468927,  1.29560752,  1.17674575]])

### Cek Outlier

In [7]:
dfZscore = pd.DataFrame(datastd)
dfZscore.head()

dfZscore[(dfZscore < -2.5) | (dfZscore > 2.5)] = np.NaN
dfZscore.isnull().sum()

0     82
1     62
2     80
3    209
4    267
dtype: int64

In [8]:
dfZscore['target'] = df['target']
dfZscore

,0,1,2,3,4,target
0,-0.996742,-0.578993,-0.802604,-0.589390,-0.747850,0
1,-0.077272,1.063130,0.439607,-1.319614,-1.108712,0
2,0.612331,0.789443,0.439607,-0.606372,-0.787946,0
3,0.382463,1.063130,0.794524,-0.045967,0.334735,0
4,-1.226609,0.515755,0.439607,-0.742228,-0.787946,0
...,...,...,...,...,...,...
9556,0.382463,-0.031619,-0.270228,-1.115831,-0.226605,0
9557,-0.996742,-0.852680,-1.334980,0.803131,0.455022,1
9558,-0.537007,2.157878,1.504359,-0.691282,0.775788,0
9559,-0.996742,1.063130,0.617065,0.021961,1.377224,0


In [9]:
dataBasket = dfZscore.dropna()
dataBasket.isnull().sum()

0         0
1         0
2         0
3         0
4         0
target    0
dtype: int64

### Splitting Data

In [10]:
x = dataBasket.drop('target',axis=1)
y = (dataBasket['target'])
# print(x.shape, y.shape) 
print(x)
# print(y)
# y.unique()

             0         1         2         3         4
0    -0.996742 -0.578993 -0.802604 -0.589390 -0.747850
1    -0.077272  1.063130  0.439607 -1.319614 -1.108712
2     0.612331  0.789443  0.439607 -0.606372 -0.787946
3     0.382463  1.063130  0.794524 -0.045967  0.334735
4    -1.226609  0.515755  0.439607 -0.742228 -0.787946
...        ...       ...       ...       ...       ...
9556  0.382463 -0.031619 -0.270228 -1.115831 -0.226605
9557 -0.996742 -0.852680 -1.334980  0.803131  0.455022
9558 -0.537007  2.157878  1.504359 -0.691282  0.775788
9559 -0.996742  1.063130  0.617065  0.021961  1.377224
9560  0.612331  0.242068  0.084689  1.295608  1.176746

[8948 rows x 5 columns]


In [11]:
xtr, xts, ytr, yts = train_test_split(
    x, y, test_size = .18,random_state=100
)
# print(xtr.shape, xts.shape)
# print(ytr.shape, yts.shape)
len(xts)
len(yts)
xts

,0,1,2,3,4
2383,-1.226609,-0.578993,-1.228505,-1.064885,-1.228999
1233,-0.996742,-0.305306,0.155673,-0.912047,-0.908233
6374,-0.766874,0.242068,0.084689,1.227680,0.855980
2019,-0.537007,0.789443,-0.092769,-1.098849,-0.707754
378,-0.766874,-1.400054,-1.618914,-0.606372,-1.028520
...,...,...,...,...,...
6690,-1.226609,1.063130,0.971983,1.244662,2.459810
7658,0.382463,0.242068,0.617065,-0.334661,-0.386988
4444,-0.077272,-0.031619,1.326900,1.822048,1.176746
4277,-0.766874,-1.400054,-0.802604,0.327636,-0.507276


# Modelling

### 1. Logistic Regression

In [12]:
logreg = LogisticRegression()
logreg.fit(xtr, ytr)
# logreg_predictions = logreg.predict(xts)
# logreg_predictions

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
logreg.score(xts,yts)

0.9826194909993793

### 2. Random Forest 

In [14]:
rfModel = RandomForestClassifier(n_estimators=100)
rfModel.fit(xtr,ytr)
# len(rfModel.estimators_) #ada 100 estimators
# rfModel_predictions = rfModel.predict(xts)
# rfModel_predictions

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
rfModel.score(xts, yts)

0.9987585350713842

### 3. K-Nearest Neighbors 

In [16]:
## nilai K
k = round(xtr.shape[0] ** .5) + 1 
k
knnmodel = KNeighborsClassifier(n_neighbors=k)
knnmodel.fit(xtr,ytr)
# knnmodel_predictions = knnmodel.predict(xts)
# knnmodel_predictions

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=87, p=2,
                     weights='uniform')

In [17]:
knnmodel.score(xts, yts)

0.9869646182495344

In [18]:
print('Score: ')
print('Logistic Regression Score = {} \nRandom Forest Score = {} \nKNN Score = {} '
      .format(logreg.score(xts,yts),rfModel.score(xts,yts), knnmodel.score(xts,yts)))

Score: 
Logistic Regression Score = 0.9826194909993793 
Random Forest Score = 0.9987585350713842 
KNN Score = 0.9869646182495344 


### Cross Validation 

In [19]:
rfModel_score = np.mean(cross_val_score(logreg,x,y, cv=5))
logreg_score = np.mean(cross_val_score(rfModel, x, y, cv=5))
knnmodel_score = np.mean(cross_val_score(knnmodel, x, y, cv=5))

print('Logistic Regression Score = {} \nRandom Forest Score = {} \nKNN Score = {} '
      .format(rfModel_score, logreg_score, knnmodel_score))

Logistic Regression Score = 0.9845773207465861 
Random Forest Score = 0.9984356292801134 
KNN Score = 0.9868129568967403 


# Hyper-parameter Tuning 

### 1. Logistic Regression

In [20]:
# param yang akan di tuned + nilai yang mungkin

penalty = ['l1','l2','elasticnet','none']
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
max_iter = [1, 10, 100, 1000, 10000]

param = {
    'penalty':penalty, 'solver':solver, 'max_iter':max_iter
}
param

# logreg.get_params()

{'penalty': ['l1', 'l2', 'elasticnet', 'none'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
 'max_iter': [1, 10, 100, 1000, 10000]}

In [21]:
# logreg.get_params()

### - Randomized Search CV

In [22]:
model = LogisticRegression()
modelrs = RandomizedSearchCV(
    estimator = model, param_distributions = param, cv = 5 
)

In [23]:
modelrs.fit(xtr,ytr)
modelrs.best_params_

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which 

{'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 10}

In [24]:
modellogreg = LogisticRegression(
    solver='lbfgs', penalty='l2', max_iter=10
)
modellogreg.fit(xtr,ytr)
modellogreg.score(xts, yts)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.983860955927995

### 2. Random Forest 

In [25]:
model = RandomForestClassifier()
#hyper parameters set
param = {'criterion':['gini','entropy'],
          'n_estimators':[10,15,20,25,30],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[3,4,5,6,7], 
          'random_state':[123],
          'n_jobs':[-1]}
modelrs = RandomizedSearchCV(
    estimator = model, param_distributions = param, cv = 5 
)

In [26]:
modelrs.fit(xtr,ytr)
modelrs.best_params_

{'random_state': 123,
 'n_jobs': -1,
 'n_estimators': 15,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'criterion': 'entropy'}

In [27]:
modelrf = RandomForestClassifier(
    random_state=123, n_jobs=-1, n_estimators=15, min_samples_split=5,min_samples_leaf=1,criterion='entropy'
)
modelrf.fit(xtr,ytr)
modelrf.score(xts, yts)

1.0

### 3. K-Nearest Neighbors

In [28]:
model = KNeighborsClassifier(n_jobs=-1)
#Hyper Parameters Set
param = {'n_neighbors':[5,6,7,8,9,10],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
modelrs = RandomizedSearchCV(
    estimator = model, param_distributions = param, cv = 5 
)

In [29]:
modelrs.fit(xtr,ytr)
modelrs.best_params_

{'weights': 'distance',
 'n_neighbors': 7,
 'n_jobs': -1,
 'leaf_size': 2,
 'algorithm': 'kd_tree'}

In [30]:
modelknn = KNeighborsClassifier(
    weights='distance', n_neighbors=7,n_jobs=-1,leaf_size=2,algorithm='kd_tree'
)
modelknn.fit(xtr,ytr)
modelknn.score(xts, yts)

0.9900682805710739

In [31]:
print('Score after Hyper-Parameter Tuning :')
print('Logistic Regression Score = {} \nRandom Forest Score = {} \nKNN Score = {} '
      .format(modellogreg.score(xts,yts), modelrf.score(xts,yts), modelknn.score(xts,yts)))

Score after Hyper-Parameter Tuning :
Logistic Regression Score = 0.983860955927995 
Random Forest Score = 1.0 
KNN Score = 0.9900682805710739 


### Prediksi

In [33]:
datatest= xts
datatest['target actual']= yts
datatest.head(5)
xts= xts.drop('target actual', axis=1)

# xts

datatest['predik LogReg']=modellogreg.predict(xts)
datatest['predik RandomForest']= modelrf.predict(xts)
datatest['predik KNN']= modelknn.predict(xts)
datatest.head(5)

<ipython-input-33-928cf7416314>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatest['target actual']= yts
<ipython-input-33-928cf7416314>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatest['predik LogReg']=modellogreg.predict(xts)
<ipython-input-33-928cf7416314>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user

,0,1,2,3,4,target actual,predik LogReg,predik RandomForest,predik KNN
2383,-1.226609,-0.578993,-1.228505,-1.064885,-1.228999,0,0,0,0
1233,-0.996742,-0.305306,0.155673,-0.912047,-0.908233,0,0,0,0
6374,-0.766874,0.242068,0.084689,1.227680,0.855980,0,0,0,0
2019,-0.537007,0.789443,-0.092769,-1.098849,-0.707754,0,0,0,0
378,-0.766874,-1.400054,-1.618914,-0.606372,-1.028520,0,0,0,0


### Evaluation Metrics

- balanced accuracy, precision, recall, F1 score & ROC AUC score.

### 1. Logistic Regression

In [34]:
tplogreg, fnlogreg, fplogreg, tnlogreg = confusion_matrix(datatest['target actual'],datatest['predik LogReg']).ravel()
tplogreg, fnlogreg,fplogreg,tnlogreg

print('recall (+) =', recall_score(datatest['target actual'],datatest['predik LogReg']))
print('recall (-) =', recall_score(datatest['target actual'],datatest['predik LogReg']))
print('Precision (+) = ', precision_score(datatest['target actual'],datatest['predik LogReg']))
print('Precision (-) = ', precision_score(datatest['target actual'],datatest['predik LogReg']))

#Hitung Balanced Accuracy
recallP = tplogreg/(tplogreg + fnlogreg)
recallN = tnlogreg/(fplogreg+tnlogreg)
bAcc= (recallP + recallN)/2
print('bAcc=', bAcc)

#Hitung F1 Score 
print('F1 Score (+) =', f1_score(datatest['target actual'],datatest['predik LogReg']))
print('F1 Score (-) =', f1_score(datatest['target actual'],datatest['predik LogReg']))

print('roc_auc_score =',roc_auc_score(datatest['target actual'],datatest['predik LogReg'].tolist(), average='macro', multi_class='ovo'))
# np.array(yts.tolist()).shape

recall (+) = 0.36363636363636365
recall (-) = 0.36363636363636365
Precision (+) =  0.7058823529411765
Precision (-) =  0.7058823529411765
bAcc= 0.6802338979145063
F1 Score (+) = 0.48000000000000004
F1 Score (-) = 0.48000000000000004
roc_auc_score = 0.6802338979145063


### 2. Random Forest 

In [35]:
tprf, fnrf, fprf, tnrf = confusion_matrix(datatest['target actual'],datatest['predik RandomForest']).ravel()
tplogreg, fnlogreg,fplogreg,tnlogreg

print('recall (+) =', recall_score(datatest['target actual'],datatest['predik RandomForest']))
print('recall (-) =', recall_score(datatest['target actual'],datatest['predik RandomForest']))
print('Precision (+) = ', precision_score(datatest['target actual'],datatest['predik RandomForest']))
print('Precision (-) = ', precision_score(datatest['target actual'],datatest['predik RandomForest']))

#Hitung Balanced Accuracy
recallP = tprf/(tprf + fnrf)
recallN = tnrf/(fprf+tnrf)
bAcc= (recallP + recallN)/2
print('bAcc=', bAcc)

#Hitung F1 Score 
print('F1 Score (+) =', f1_score(datatest['target actual'],datatest['predik RandomForest']))
print('F1 Score (-) =', f1_score(datatest['target actual'],datatest['predik RandomForest']))

print('roc_auc_score =',roc_auc_score(datatest['target actual'],datatest['predik RandomForest'].tolist(), average='macro', multi_class='ovo'))
# np.array(yts.tolist()).shape

recall (+) = 1.0
recall (-) = 1.0
Precision (+) =  1.0
Precision (-) =  1.0
bAcc= 1.0
F1 Score (+) = 1.0
F1 Score (-) = 1.0
roc_auc_score = 1.0


### 3. K-Nearest Neighbors 

In [36]:
tpknn, fnknn, fpknn, tnknn = confusion_matrix(datatest['target actual'],datatest['predik KNN']).ravel()
tpknn, fnknn,fpknn,tnknn

print('recall (+) =', recall_score(datatest['target actual'],datatest['predik KNN']))
print('recall (-) =', recall_score(datatest['target actual'],datatest['predik KNN']))
print('Precision (+) = ', precision_score(datatest['target actual'],datatest['predik KNN']))
print('Precision (-) = ', precision_score(datatest['target actual'],datatest['predik KNN']))

#Hitung Balanced Accuracy
recallP = tprf/(tprf + fnrf)
recallN = tnrf/(fprf+tnrf)
bAcc= (recallP + recallN)/2
print('bAcc=', bAcc)

#Hitung F1 Score 
print('F1 Score (+) =', f1_score(datatest['target actual'],datatest['predik KNN']))
print('F1 Score (-) =', f1_score(datatest['target actual'],datatest['predik KNN']))

print('roc_auc_score =',roc_auc_score(datatest['target actual'],datatest['predik KNN'].tolist(), average='macro', multi_class='ovo'))
# np.array(yts.tolist()).shape

recall (+) = 0.6060606060606061
recall (-) = 0.6060606060606061
Precision (+) =  0.8695652173913043
Precision (-) =  0.8695652173913043
bAcc= 1.0
F1 Score (+) = 0.7142857142857143
F1 Score (-) = 0.7142857142857143
roc_auc_score = 0.8020797326880977


### Prediksi Pemain

In [37]:
dataPemain = pd.read_csv('pemainIndo.csv', delimiter=';')
dataPemain

,Name,Club,Country,Age,Height,Weight,Avg Points,Avg Rebounds
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5


### Standarisasi Data

In [38]:
dataPemaintest = dataPemain[[' Age',' Height', ' Weight', ' Avg Points', ' Avg Rebounds']]

stdscalertest = StandardScaler()

datastdtest = stdscalertest.fit_transform(dataPemaintest)
datastdtest

array([[ 0.63599873,  1.32911403,  1.31632171,  0.14285714,  0.48349378],
       [-0.95399809,  0.25724788,  0.42390021, -0.21428571, -0.72524067],
       [ 0.10599979, -1.24336474, -0.24541591,  1.21428571, -0.72524067],
       [ 1.69599661,  0.25724788, -2.0302589 ,  1.57142857, -0.32232919],
       [-1.48399703, -0.17149859, -0.91473203, -0.57142857, -1.12815215],
       [-0.95399809,  1.54348727,  0.20079484, -0.92857143,  2.09513971],
       [ 1.16599767,  0.90036757,  1.31632171,  0.14285714,  0.0805823 ],
       [ 0.10599979, -1.02899151,  0.64700558, -2.        , -1.12815215],
       [-0.95399809, -1.45773797, -0.91473203,  0.85714286,  1.28931674],
       [ 0.63599873, -0.38587182,  0.20079484, -0.21428571,  0.0805823 ]])

In [39]:
dataPemain['predict'] = modelrf.predict(datastdtest)
dataPemain

,Name,Club,Country,Age,Height,Weight,Avg Points,Avg Rebounds,predict
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6,0
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3,0
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3,0
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4,0
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2,0
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10,0
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5,0
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2,0
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8,1
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5,0


In [40]:
dataPemain['keterangan']=dataPemain.apply(lambda x: 'tidak lolos' if (x['predict']== 0) else ' lolos', axis=1)
dataPemain

,Name,Club,Country,Age,Height,Weight,Avg Points,Avg Rebounds,predict,keterangan
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6,0,tidak lolos
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3,0,tidak lolos
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3,0,tidak lolos
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4,0,tidak lolos
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2,0,tidak lolos
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10,0,tidak lolos
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5,0,tidak lolos
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2,0,tidak lolos
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8,1,lolos
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5,0,tidak lolos
